In [8]:
# Load all required libraries for this notebook.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Loading specific modules specific Important additional statistical analysis functionality
from collections import Counter
from scipy import stats
from scipy.stats import norm
from pandas.api.types import infer_dtype #added to help check for mixed data type during checking for formatting issues within each column while data cleaning

# Important functionality for this lesson ---> Statistical tools for model building etc
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Important functionality for this lesson
from functools import lru_cache
from functools import wraps
import pprint as pp
from time import time

import warnings
warnings.filterwarnings('ignore')

# Setting globally figure size in the notebook
plt.rcParams["figure.figsize"] = (8.0, 6.0)

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
%%time

#Loading the data and Preserve 'NA' and 'None' as strings
complaints_df = pd.read_csv('gdrive/MyDrive/data/complaints.csv')

#Loading the data and Preserve 'NA' and 'None' as strings
#complaints_with_None_NA_df = pd.read_csv('gdrive/MyDrive/data/complaints.csv', na_filter=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/complaints.csv'

In [7]:
# Making a copy of the original data before preprocessing so you can refer back to it without reloading the CSV file
df = complaints_df.copy() #df_with_None_NA_df = complaints_with_None_NA_df.copy()

# Summary info of the data
df.info()

# Viewing the dataset
df.head(10)

NameError: name 'complaints_df' is not defined

In [ ]:
#Viewing count of non-missing values using '.count()' because '.info()' dindt show the non-null count given the large size of the data
print(df.count())

## Part 1 - Data Cleaning and Preparation

#### &nbsp; (1) Resolve data type inconsistencies

In [ ]:
# Converting date-related columnsto datetime format
df['Date received'] = pd.to_datetime(df['Date received'], errors= 'coerce') # errors='coerce' is used so tha any value that cannot be successfully parsed into a valid datetime object will be replaced with NaT (Not a Time). NaT is the pandas equivalent of NaN (Not a Number) for datetime objects, representing a missing or invalid datetime value.
df['Date sent to company'] = pd.to_datetime(df['Date sent to company'], errors= 'coerce')

#Viewing data and data info to confirm datatype change to datetime format
df.info()
df.head()

#### &nbsp; (2a) Check for formating issues within columns with object data types

In [ ]:
# Creating a list of df columns that are objects
obj_columns_list = list(df.select_dtypes(include='object').columns)

#Checking if each object column has mixed  data type
count = 0 #initiate counter to monitor number of object columns with mixed data type
for col in obj_columns_list:
    dat_type = infer_dtype(df[col])
    if dat_type == 'mixed':
        count = + 1
        print(f"'{col}' is a column with objects of mixed data types")
    else:
        print(f"'{col}' is a column with objects of only '{dat_type}' data types")

# Printing confirmation of no mixed data type if counter = 0
if count == 0:
    print(f"\nNone of the columns with objects have mixed data types")

#### &nbsp; (2b) Resolve formating issues

In [ ]:
# Striping whitespaces from all object columns since they are all string data types ensure that there are no extra spaces at the beginning or end of the strings
df[obj_columns_list] = df[obj_columns_list].apply(lambda x: x.str.strip(), axis=0 )
#FYI: Other string casings issues to consider include adding '.str.lower()' , '.str.upper()' , '.str.title()'.

# Quick visual Inspection of object columns
df[obj_columns_list]

#### &nbsp; (3) Check for duplicate entries

In [ ]:
# Total count of duplicate row entries, excluding the first occurrence of the duplicate entry
duplicate_count = df.duplicated(keep='first').sum()

# proportion of duplicate entries, excluding the first occurrence of the duplicate entry
duplicate_proportion = df.duplicated(keep='first').mean()*100

print(f"\nThe data contains {duplicate_count} rows that are duplicate entries (excluding the first occurence), which is {round(duplicate_proportion, 2)}% of the total dataset\n")

In [ ]:
# #Visual inspection of duplicate row/entries (with the inclusion of the first occurence)
# df[df.duplicated(keep=False)].sort_values('Date received').head(20)

In [ ]:
# # Removing duplicate rows while retaining the first occurrence
# df = df.drop_duplicates(keep='first')

# # Checking data info & size after removing duplicates
# df.info()

#### &nbsp; (3) Check for Missing/Empty data (NaN, Null, None, and Empty Strings)

In [ ]:
# Creating a dataframe boolean mask to identify all missing values and empty strings in each column
df_mask_missingdata = df.isna() | df.eq('')

# Total count of missing/empty data for each column/variable
missingdata_count = df_mask_missingdata.sum()

# Proportion of missing/empty data for each column/variable
missingdata_proportion = df_mask_missingdata.mean() *100

print(f" The count of mising/empty values per variable is \n\n{missingdata_count} \n")

print(f" The proportion of mising/empty values per variable is \n\n{round(missingdata_proportion,2)} \n")

#Creating a Series boolean mask for missing data in each row
rows_with_missingdata =  df_mask_missingdata.any(axis=1)

#Visual inspection of row entries with 1 or more missing/empty data
df[rows_with_missingdata].head(10)

### Clean the Missing data identified in each variable/column

#### &nbsp; (1) Company public response: Replace missing values with 'None' to replicate original data value that Panda removed

In [ ]:
# Viewing the categories within this variable
df['Company public response'].value_counts()

In [ ]:
# Replacing missing values with 'None', which is the actual category in the original dataset, but was removed when loaded with panda
df['Company public response'] = df['Company public response'].fillna('None')
assert df['Company public response'].isna().sum() == 0

In [ ]:
# Viewing the categories within this variable again to confirm missing value was replaced
df['Company public response'].value_counts()

#### &nbsp; (2) State: Replace missing values with 'Unknown'

In [ ]:
# Viewing the categories within this variable
df['State'].value_counts()

In [ ]:
# Replacing missing values with 'Unknown'
df['State'] = df['State'].fillna('Unknown')
assert df['State'].isna().sum() == 0

In [ ]:
# Viewing the df to confirm missing value for the variable was replaced
df[df['State'] == 'Unknown'].head(5)

#### &nbsp; (3) ZIP code: Replace missing values with 'Unknown'

In [ ]:
# Viewing the categories within this variable
df['ZIP code'].value_counts()

In [ ]:
# Replacing missing values with 'Unknown'
df['ZIP code'] = df['ZIP code'].fillna('Unknown')
assert df['ZIP code'].isna().sum() == 0

In [ ]:
# Viewing the categories within this variable again to confirm missing value was replaced
df['ZIP code'].value_counts()

In [ ]:
# Replacing 'XXXXX' values with 'Unknown'
df['ZIP code'].replace('XXXXX', 'Unknown', inplace=True)

# Viewing the categories within this variable again to confirm XXXXX value was replaced
df['ZIP code'].value_counts()

#### &nbsp; (4) Tags: Replace missing values with 'None' to replicate original data value that Panda removed

In [ ]:
# Viewing the categories within this variable
df['Tags'].value_counts()

In [ ]:
# Replacing missing values with 'None', which is the actual category in the original dataset, but was removed when loaded with panda
df['Tags'] = df['Tags'].fillna('None')
assert df['Tags'].isna().sum() == 0

In [ ]:
# Viewing the categories within this variable again to confirm missing value was replaced
df['Tags'].value_counts()

#### &nbsp; (5) Consumer consent provided?: Replace missing values with 'None' to replicate original data value that Panda removed

In [ ]:
# Viewing the categories within this variable
df['Consumer consent provided?'].value_counts()

In [ ]:
# Replacing missing values with 'None', which is the actual category in the original dataset, but was removed when loaded with panda
df['Consumer consent provided?'] = df['Consumer consent provided?'].fillna('None')
assert df['Consumer consent provided?'].isna().sum() == 0

In [ ]:
# Viewing the categories within this variable again to confirm missing value was replaced
df['Consumer consent provided?'].value_counts()

#### &nbsp; (6) Consumer disputed?: Replace missing values with 'Dispute Tracking Discontinued after 24April2017' to replicate original data value that Panda removed

In [ ]:
# Viewing the categories within this variable
df['Consumer disputed?'].value_counts()

In [ ]:
# Replacing missing values with ''Consumer complaint narrative'.  The actual category in the original dataset was N/A, but was removed when loaded with panda.  The Dataset library indicated N/A means The Bureau discontinued the consumer dispute option on April 24, 2017.
df['Consumer disputed?'] = df['Consumer disputed?'].fillna('Dispute Tracking Discontinued after 24April2017')
assert df['Consumer disputed?'].isna().sum() == 0

In [ ]:
# Viewing the categories within this variable again to confirm missing value was replaced
df['Consumer disputed?'].value_counts()

#### &nbsp; (7) Consumer complaint narrative: Replace missing values with 'Consumer did not consent to providing narrative'

In [ ]:
# Viewing the categories within this variable
df['Consumer complaint narrative'].value_counts()

In [ ]:
# Replacing missing values with 'Notprovided'
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].fillna('Consumer did not consent to providing narrative')
assert df['Consumer complaint narrative'].isna().sum() == 0

In [ ]:
# Viewing the df to confirm missing value for the variable was replaced
df[df['Consumer complaint narrative'] == 'Consumer did not consent to providing narrative'].head(5)

#### &nbsp; (8) Sub-issue: Replace missing values with 'None' to replicate original data value that Panda removed

In [ ]:
# Viewing the categories within this variable
df['Sub-issue'].value_counts()

In [ ]:
# Replacing missing values with 'None', which is the actual category in the original dataset, but was removed when loaded with panda
df['Sub-issue'] = df['Sub-issue'].fillna('None')
assert df['Sub-issue'].isna().sum() == 0

In [ ]:
# Viewing the df to confirm missing value for the variable was replaced
df[df['Sub-issue'] == 'None'].head(5)

#### &nbsp; (9) Sub-product: Replace missing values with 'Unknown'

In [ ]:
# Viewing the categories within this variable
df['Sub-product'].value_counts()

In [ ]:
# Replacing missing values with with 'Unknown'
df['Sub-product'] = df['Sub-product'].fillna('Unknown')
assert df['Sub-product'].isna().sum() == 0

In [ ]:
# Viewing the categories within this variable again to confirm missing value was replaced
df['Sub-product'].value_counts()

In [ ]:
# Viewing the df to confirm missing value for the variable was replaced
df[df['Sub-product'] == 'Unknown'].head(5)

In [ ]:
# Checking data info for cleaned data to confirm there are no missing values in all the columns. Here i am using '.count()' because '.info()' dindt show the non-null count given the large size of the data
print(df.count())

### Save the Cleaned Data

In [ ]:
#save cleaned data file
df.to_csv('data/complaints_clean.csv', index=False)

## Part 2 - Basic EDA